## 评估数据：结构（乱数据-整洁数据）+内容（脏数据-干净数据）——整洁度、干净度
       整洁数据：根据埃德加科德第三范式，有三个特点：每列是一个变量；每行是一个观察值；每个单元格是一个值
       列是数据，行是实例
       整洁数据并不一定更加直观，他是为了更利于数据分析
       脏数据：丢失数据，空缺；重复数据；不一致数据；无效或错误数据
       数据结构比数据内容更好评估
       

### 评价结构 info head tail sample 
dataframe info() 概况信息（列名、列对应的数据类型）——快速了解信息
通过 head（） tail（）方法是否符合三范式；sample（）——随机几行
调整展示上限：pd.set_option("display.max_columns",150)
         pd.set_option("display.max_colwidth",500)

### 评价内容 info isnull sum
#### 评估丢失数据：
    info（）列出每列非空缺值数量；series的isnull方法，各个值是否为空缺，用在dataframe上，则是生成由布尔值组成的dataframe表格。连续调用isnull（）.sum()——得到空缺的数量（因为布尔值T为1，F为0，计入运算）
    so 计算某列空缺值个数：scores["考试1"].isnull().sum()
       计算每列空缺值个数：scores.isnull().sum()
   
    提取有空缺的行 or 列(根据条件提取)：
    scores[scores["考试3"].isnull()]
#### 评估重复数据：duplicated（subset=["",""]）
    students["学号"].duplicated()——某值第一次出现为F，第二次为T
    对于SEries
    对于dataframe——只有行的信息完全一样，才会显示T
    对于多个属性进行：可选参数subset：students.duplicated(subset=["学号","性别"])
    提取有问题的行：students[students.duplicated(subset=["学号","性别"])]
#### 评估不一致数据：value_counts（）
    借用series的value_counts()方法——可以看到dataframe某列下面所有值的种类，以及各个种类出现的次数——确定有没有不同值在表示同一目标的情况
    students["班级"].value_counts（）
#### 评估无效/错误数据：sort_values（），describe()
    特别离谱的——排序方法：异常小值，异常大值——students["身高"].sort_values（）
    students.describe() 展示最小值最大值，是否有异常数据存在
#### 其他评估技能
#### 评估目的：基于评估结果，确认下一步行动

In [1]:
True+3

4

In [2]:
False+2

2

## 清洗数据 清洗之术-pandas库-道
    先结构再内容：
    结构：转置行列、拆分、重塑、多个观察值拆分
    内容：丢失数据（人工填入；不处理；如果是关键变量缺失，删除当行；值填充）
        重复数据：删除
        不一致数据：统一，统一含义只保留一种表达方式
        无效数据：删除or替换，pandas自动忽略nan，so，反而不影响，or值填充
        变量转换：具体变量的数据类型之间的转换（手机号，int64-str）
    利用pandas库进行实际的数据清理。

### pandas库的清洗之道 rename
#### 清理索引和列名 rename set_index reset_index sort_index
    dataframe的rename方法——重命名索引和列名
        df.rename(index={...})——修改索引
            df.rename(index={"2-":"2","_5":"5"})
        df.rename(columns={...})——修改列名
            df.rename(columns={"客户_姓名":"客户姓名","客户 性别":"客户性别"})
        tips：并不改变原始值，需要赋值，or，inplace=True
             df.rename(columns={...}，inplace=True)
             
    rename()可传入函数or方法！
        df.rename(index=某函数/方法)——修改索引
           df.rename(columns=str.upper)
           tips:
               pandas官方文档（61课，2分40秒）
               series、dataframe方法相关文档
        df.rename(columns=某函数/方法)——修改列名
    
    把某列设为索引——df.set_index("Salesperson")；reset_index
        df.set_index("Salesperson")——将某列设为索引
        df.reset_index("Salesperson")——将设为原本默认（与上相反）
    
    对索引和列名重新排序：sort_index(),默认axis=0，沿着索引纵向
        不改变原始值，赋值or inplace=Ture
        
#### 针对乱数据的清理 T  ；str.splitt("/",expand=True)  drop；str.cat；pd.melt；explode（）——一般都不改变原始值——赋值orinplace=True
    对数据转置：df = df.T
    
    对列进行拆分：一列变多列
        对单个字符串:"28.53/0.87".split("/")——['28.53','0.87']
        对列进行拆分：str.split（series的方法）
            df["人口密度"].str.split("/",expand=True),expand=True表示分割为单独的series
            df["人口","面积"]=df["人口密度"].str.split("/",expand=True)
            df = df.drop("人口密度"，axis=1)——将拆分前的列删除
            
    对列进行合并：str.cat
        df["姓名"] = df["姓"].str.cat(df["名"],sep="_"),sep="_"指是以_为链接
        df = df.drop(["姓","名"]，axis=1)
        
    把宽数据转换为长数据 ：pd.melt 
         不改变原始dataframe，重新赋值
         pd.melt(df,id_vars=['国家代码','年份']，var_name='年龄组'，value_name='肺结核病例数')
             df——对象
             id_vars——不变列
             var_name——新列
             value_name——新列
     
     对行进行拆分：df.explode("课程列表")
     
     对行或列进行删除：drop，默认axis=0，为行
         不改变原始值
         df.drop([2,4])
         df.drop(["考试2","考试4"]，axis=1)

#### 针对脏数据的清理loc，iloc，fillna（分series逐列，dataframe传入对象），dropna(subset=[""])，drop_duplicates(subset=[""]，keep='')，repalce，——不改原始值——赋值/inplace
    对整列缺失值填充：针对列的赋值操作：
        df["国家"] = "中国"
    对某个缺失值填充：loc、iloc定位填充：
        df.loc["003","销售额"] = 800
    对部分缺失值填充：loc、iloc定位填充：
        df.loc["003":"004","日期"] = "2005-01-03"   
    自动找到缺失并填充:df["B"].fillna(0)——自动找到NAN，并填充为0
        df["B"].fillna(df["B".mean()])——自动找到NAN，并填充为平均值
        df.fillna({'A':0,'B':10.'C':20})——对dataframe使用，传入对象，键为列名，值为对应列NAN填数——更高效
     
     删除存在缺失值的行：如果缺失关键数据——不得不丢弃观察值——dropna
         df.dropna(subset=["工资"]，axis=0)——仅删除缺少“工资”行,默认axis=0——行,
     删除重复数据：drop_duplicates ——删除series重复值，or dataframe的重复行 
         df["姓名"].drop_duplicates()——删除重复值
         df.drop_duplicates()——删除重复行
         df.drop_duplicates(subset=["姓名","学号"]，keep='last',axis=0)——删除姓名和性别重复的行——！！是同时重复！！——删除的是第一个;keep='last'——表示保留最后一个重复值
         
     对值进行替换:repalce
         series调用：df["学校"].repalce("清华","清华大学")
            df["学校"].repalce(["清华","五道口"],"清华大学") 
             ——将清华转为清华大学
         dataframe调用：df.repalce("清华","清华大学")
             前者可以是数组、也可以是对象
             repalce_dict={'华南':'华南理工大学'，"清华"："清华大学"}
             df.replace(repalce_dict)
     对值的类型进行替换：series的astype（）方法;type
         s1.astype(int)
         s1.astype(str)
         s1.astype(float)
         s1.astype(bool)
         type()——返回类型名称
         tips：在python里面str以object名称表示
         
         pandas库的category（）方法，分类数据
         数据的类型：分类数据（有限数量的不同类别数据：如性别、奖牌）、数值数据（具体的测量数值，不受限制，譬如0到1之间有无数的数字——进行数学运算是有意义的）
         分类数据——当是有限种类的，pandas推荐将其转换为category数据类型——节约内存，可视化分析时选择合适的统计方法或图表类型
         s1.astype("category")

### 保存清洗的数据
    写入csv——df.to_csv("cleaned_sales_data.csv")
        调用后将dataframe转换为csv格式，并保存到路径对应的文件
        但是由dataframe转化而来的csv，再转化为 dataframe 就会有一个多余的unnamed：0
        rename（）——cleaned_df.rename(columns={"Unnamed:0":"销售ID"}，inplace = True)
        cleaned_df.set_index("销售ID"，inplace = True)
        
     or,直接不读取索引（如果无关键信息）——df.to_csv("cleaned_sales_data.csv"，index=False)